<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
</center>
<center>Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

# <center> Capstone проект №1. Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 5.  Соревнование Kaggle "Catch Me If You Can"

На этой неделе мы вспомним про концепцию стохастического градиентного спуска и опробуем классификатор Scikit-learn SGDClassifier, который работает намного быстрее на больших выборках, чем алгоритмы, которые мы тестировали на 4 неделе. Также мы познакомимся с данными [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) Kaggle по идентификации пользователей и сделаем в нем первые посылки. По итогам этой недели дополнительные баллы получат те, кто попадет в топ-30 публичного лидерборда соревнования.

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
**Также рекомендуется вернуться и просмотреть [задание](https://www.coursera.org/learn/supervised-learning/programming/t2Idc/linieinaia-rieghriessiia-i-stokhastichieskii-ghradiientnyi-spusk) "Линейная регрессия и стохастический градиентный спуск" 1 недели 2 курса специализации.**

In [1]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score
from scipy import sparse

**Считаем данные [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [2]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = ('data')

In [3]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id')

In [4]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


**Объединим обучающую и тестовую выборки – это понадобится, чтоб вместе потом привести их к разреженному формату.**

In [5]:
train_test_df = pd.concat([train_df, test_df])

В обучающей выборке видим следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - user_id – ID пользователя
    
Сессии пользователей выделены таким образом, что они не могут быть длинее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут. 

**Посмотрим на статистику признаков.**

Пропуски возникают там, где сессии короткие (менее 10 сайтов). Скажем, если человек 1 января 2015 года посетил *vk.com* в 20:01, потом *yandex.ru* в 20:29, затем *google.com* в 20:33, то первая его сессия будет состоять только из двух сайтов (site1 – ID сайта *vk.com*, time1 – 2015-01-01 20:01:00, site2 – ID сайта  *yandex.ru*, time2 – 2015-01-01 20:29:00, остальные признаки – NaN), а начиная с *google.com* пойдет новая сессия, потому что уже прошло более 30 минут с момента посещения *vk.com*.

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 1 to 253561
Data columns (total 21 columns):
site1     253561 non-null int64
time1     253561 non-null object
site2     250098 non-null float64
time2     250098 non-null object
site3     246919 non-null float64
time3     246919 non-null object
site4     244321 non-null float64
time4     244321 non-null object
site5     241829 non-null float64
time5     241829 non-null object
site6     239495 non-null float64
time6     239495 non-null object
site7     237297 non-null float64
time7     237297 non-null object
site8     235224 non-null float64
time8     235224 non-null object
site9     233084 non-null float64
time9     233084 non-null object
site10    231052 non-null float64
time10    231052 non-null object
target    253561 non-null int64
dtypes: float64(9), int64(2), object(10)
memory usage: 42.6+ MB


In [7]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,2014-11-04 10:03:21,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,2014-05-16 15:05:40,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44


In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82797 entries, 1 to 82797
Data columns (total 20 columns):
site1     82797 non-null int64
time1     82797 non-null object
site2     81308 non-null float64
time2     81308 non-null object
site3     80075 non-null float64
time3     80075 non-null object
site4     79182 non-null float64
time4     79182 non-null object
site5     78341 non-null float64
time5     78341 non-null object
site6     77566 non-null float64
time6     77566 non-null object
site7     76840 non-null float64
time7     76840 non-null object
site8     76151 non-null float64
time8     76151 non-null object
site9     75484 non-null float64
time9     75484 non-null object
site10    74806 non-null float64
time10    74806 non-null object
dtypes: float64(9), int64(1), object(10)
memory usage: 13.3+ MB


**В обучающей выборке – 2297 сессий одного пользователя (Alice) и 251264 сессий – других пользователей, не Элис. Дисбаланс классов очень сильный, и смотреть на долю верных ответов (accuracy) непоказательно.**

In [9]:
train_df['target'].value_counts()

0    251264
1      2297
Name: target, dtype: int64

**Пока для прогноза будем использовать только индексы посещенных сайтов. Индексы нумеровались с 1, так что заменим пропуски на нули.**

In [10]:
train_test_df_sites = train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype(int)

In [11]:
train_test_df_sites.head(10)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,718,0,0,0,0,0,0,0,0,0
2,890,941,3847,941,942,3846,3847,3846,1516,1518
3,14769,39,14768,14769,37,39,14768,14768,14768,14768
4,782,782,782,782,782,782,782,782,782,782
5,22,177,175,178,177,178,175,177,177,178
6,570,21,570,21,21,0,0,0,0,0
7,803,23,5956,17513,37,21,803,17514,17514,17514
8,22,21,29,5041,14422,23,21,5041,14421,14421
9,668,940,942,941,941,942,940,23,21,22


**Создайте разреженные матрицы *X_train_sparse* и *X_test_sparse* аналогично тому, как мы это делали ранее. Используйте объединенную матрицу *train_test_df_sites*, потом разделите обратно на обучающую и тестовую части.**

Обратите внимание на то, что в  сессиях меньше 10 сайтов  у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить.

**Выделите в отдельный вектор *y* ответы на обучающей выборке.**

In [12]:
train_test_sparse = csr_matrix((np.ones(train_test_df_sites.values.size), 
                                train_test_df_sites.values.ravel(), 
                                np.arange(train_test_df_sites.values.shape[0] + 1) * 
                                train_test_df_sites.values.shape[1]), dtype=int)[:, 1:]

X_train_sparse = train_test_sparse[:train_df.shape[0]]
X_test_sparse = train_test_sparse[train_df.shape[0]:]
y = train_df['target'].values

**1. Выведите размерности матриц *X_train_sparse* и *X_test_sparse* – 4 числа на одной строке через пробел: число строк и столбцов матрицы *X_train_sparse*, затем число строк и столбцов матрицы *X_test_sparse*.**

In [13]:
X_train_sparse.shape, X_test_sparse.shape

((253561, 48371), (82797, 48371))

**Сохраним в pickle-файлы объекты *X_train_sparse*, *X_test_sparse* и *y* (последний – в файл *kaggle_data/train_target.pkl*).**

In [14]:
with open(os.path.join(PATH_TO_DATA, 'X_train_sparse.pkl'), 'wb') as X_train_sparse_pkl:
    pickle.dump(X_train_sparse, X_train_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'X_test_sparse.pkl'), 'wb') as X_test_sparse_pkl:
    pickle.dump(X_test_sparse, X_test_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'train_target.pkl'), 'wb') as train_target_pkl:
    pickle.dump(y, train_target_pkl, protocol=2)

**Разобьем обучающую выборку на 2 части в пропорции 7/3, причем не перемешивая. Исходные данные упорядочены по времени, тестовая выборка по времени четко отделена от обучающей, это же соблюдем и здесь.**

In [15]:
train_share = int(.7 * X_train_sparse.shape[0])
X_train, y_train = X_train_sparse[:train_share, :], y[:train_share]
X_valid, y_valid  = X_train_sparse[train_share:, :], y[train_share:]

**Создайте объект `sklearn.linear_model.SGDClassifier` с логистической функцией потерь и параметром *random_state*=17. Остальные параметры оставьте по умолчанию, разве что *n_jobs*=-1 никогда не помешает. Обучите  модель на выборке `(X_train, y_train)`.**

In [16]:
sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17)
sgd_logit.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=17, shuffle=True,
       tol=None, verbose=0, warm_start=False)

**Сделайте прогноз в виде предсказанных вероятностей того, что это сессия Элис, на отложенной выборке *(X_valid, y_valid)*.**

In [17]:
logit_valid_pred_proba = sgd_logit.predict_proba(X_valid)

**2. Напечатайте ROC AUC логистической регрессии, обученной с помощью стохастического градиентного спуска, на отложенной выборке. Округлите до 3 знаков после разделителя.**

In [18]:
print('ROC AUC =', round(roc_auc_score(y_valid, logit_valid_pred_proba[:, 1]), 3))

ROC AUC = 0.934


**Сделайте прогноз в виде предсказанных вероятностей отнесения к классу 1 для тестовой выборки с помощью той же *sgd_logit*, обученной уже на всей обучающей выборке (а не на 70%).**

In [19]:
%%time
sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17)
logit_test_pred_proba = sgd_logit.fit(X_train_sparse, y).predict_proba(X_test_sparse)[:, 1]

Wall time: 704 ms


**Запишите ответы в файл и сделайте посылку на Kaggle. Дайте своей команде (из одного человека) на Kaggle говорящее название – по шаблону "[YDF & MIPT] Coursera_Username", чтоб можно было легко идентифицировать Вашу посылку на [лидерборде](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard/public).**

**Результат, который мы только что получили, соответствует бейзлайну "SGDCLassifer" на лидерборде, задача на эту неделю – как минимум его побить.**

In [20]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [21]:
write_to_submission_file(logit_test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_sgd_logit.csv'))

Команда на Kaggle - **[YDF & MIPT] Dmitry Shereshevskiy**

## Критерии оценки работы:
- Правильные ли получились размерности матриц в п. 1? (max. 2 балла)
- Правильным ли получилось значения ROC AUC в п. 2? (max. 4 балла)
- Побит ли бенчмарк "sgd_logit_benchmark.csv" на публичной части рейтинга в соревновании Kaggle? (max. 2 балла)
- Побит ли бенчмарк "Logit +3 features" на публичной части рейтинга в соревновании Kaggle? (max. 2 балла)

## Пути улучшения
На этой неделе дается много времени на соревнование. Не забывайте вносить хорошие идеи, к которым Вы пришли по ходу соревнования, в описание финального проекта (`html`, `pdf` или `ipynb`).
Что можно попробовать:
 - Использовать ранее построенные признаки для улучшения модели (проверить их можно на меньшей выборке по 150 пользователям, отделив одного из пользователей от остальных – это быстрее)
 - Настроить параметры моделей (например, коэффициенты регуляризации)
 - Если позволяют мощности (или хватает терпения), можно попробовать смешивание (блендинг) ответов бустинга и линейной модели. [Вот](http://mlwave.com/kaggle-ensembling-guide/) один из самых известных тьюториалов по смешиванию ответов алгоритмов, также хороша [статья](https://alexanderdyakonov.wordpress.com/2017/03/10/cтекинг-stacking-и-блендинг-blending) Александра Дьяконова
 - Обратите внимание, что в соревновании также даны исходные данные о посещенных веб-страницах Элис и остальными 1557 пользователями (*train.zip*). По этим данным можно сформировать свою обучающую выборку. 

На 6 неделе мы пройдем большой тьюториал по Vowpal Wabbit и попробуем его в деле, на данных соревнования.

# Работа по побитию бенчмарков на Kaggle

Для начала посмотрим на словарь, который имеется на Kaggle:

In [22]:
site_freq_path=os.path.join(PATH_TO_DATA, 'site_dic.pkl')
with open(site_freq_path, 'rb') as pkl_file:
            freq_vocabulary = pickle.load(pkl_file)

In [23]:
dict(list(freq_vocabulary.items())[:5])

{'cdt46.media.tourinsoft.eu': 30911,
 'groups.live.com': 13997,
 'majeureliguefootball.wordpress.com': 42436,
 'www.abmecatronique.com': 25075,
 'www.hdwallpapers.eu': 8104}

Видим, что формат словаря **немного другой** (без частот), примем к сведению.

Подготовим код для рассчета частотного словаря.    
ПРИМЕЧАНИЕ: запустил было этот код, но неожидано о-о-о-очень долго считает (срубил через 20 мин., прогнозно - еще 20 мин., но скорость вычислений падает, похоже - мало памяти)

Сделаем функцию, которая принимает на вход **train_df**, а выдает **X_train_sparse** с признаками-**сайтами** в виде "мешка слов" и новыми признаками (см. коммент в теле функции),    
ну и плюс целевой признак **y**.

In [29]:
def feature_engineering_df(train_df, features, train=True):
    '''
    features = ['#unique_si tes', 'session_timespan', 'start_hour', 'day_of_week', 'day_of_week', 'weekend']
    '''
    # повторим, что было выше (частично)
    train_df_sites = train_df[['site%d' % i for i in range(1, 11)]] # не забыть заполнить нули
    
    # подготовим временнЫе метки
    train_df[['time%d' % i for i in range(1, 11)]] = train_df[['time%d' % i for i in range(1, 11)]].applymap(pd.to_datetime)
    timestamp_df = train_df[['time%d' % i for i in range(1, 11)]].applymap(lambda x: pd.datetime.timestamp(x) 
                                                                  if type(x) == pd._libs.tslib.Timestamp else np.nan)
    # добавляем признаки
    if '#unique_sites' in features:
        train_df['#unique_sites'] = train_df[['site' + str(i) for i in range(1, 11)]].nunique(axis=1, dropna=True)
    if 'session_timespan' in features:
        train_df['session_timespan'] = timestamp_df.max(axis=1, skipna=True) - timestamp_df.min(axis=1, skipna=True)
    if 'start_hour' in features:    
        train_df['start_hour'] = train_df['time1'].apply(lambda x: x.hour)
    if 'day_of_week' in features:    
        train_df['day_of_week'] = train_df['time1'].apply(lambda x: x.weekday())
    if 'time_of_day' in features:    
        # время суток (06-12 - утро(1), 12-19 - день(2), 19-24 - вечер(3), 00-06 - ночь(4))
        train_df['time_of_day'] = train_df['time1'].apply(lambda x:  1 if x.hour >  6 and x.hour <= 12 else 
                                                                     2 if x.hour > 12 and x.hour <= 19 else
                                                                     3 if x.hour > 19 and x.hour <= 24 else 
                                                                     4)
    if 'weekend' in features:
        # индикатор уикэнда в день начала сессии
        train_df['weekend'] = train_df['time1'].apply(lambda x: 1 if x.weekday() in [5, 6] else 0)
   
    train_df_sites = train_df_sites.fillna(0).astype('int')
    train_sparse = csr_matrix((np.ones(train_df_sites.values.size), 
                                train_df_sites.values.ravel(), 
                                np.arange(train_df_sites.values.shape[0] + 1) * 
                                train_df_sites.values.shape[1]), dtype=int)[:, 1:]
    if train:
        y = train_df['target'].values
    else:
        y = None

    return train_sparse, train_df[features], y

In [25]:
def logit_roc_auc_score(X_train_sparse, y):
    '''
    обучение и расчет метрики на отложенной выборке
    '''

    train_share = int(.7 * X_train_sparse.shape[0])
    X_train, y_train = X_train_sparse[:train_share, :], y[:train_share]
    X_valid, y_valid  = X_train_sparse[train_share:, :], y[train_share:]

    sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17)
    sgd_logit.fit(X_train, y_train)

    logit_valid_pred_proba = sgd_logit.predict_proba(X_valid)

    return roc_auc_score(y_valid, logit_valid_pred_proba[:, 1])

**Сохраним в pickle-файлы объекты X_train_sparse.**

Проверим **все** комбинации добавления признаков:

In [26]:
import itertools

In [30]:
features = ['#unique_sites', 'session_timespan', 'start_hour', 'day_of_week', 'time_of_day', 'weekend']
train_sites_sparse, features_df, y_values = feature_engineering_df(train_df, features, train=True)

feature_scores = []
for r in range(len(features) + 1):
    for fe in itertools.combinations(features, r):
        fe_ = list(fe)
        features_sparse = csr_matrix(features_df[fe_].values)
  
        X_train_sparse = sparse.hstack([train_sites_sparse , features_sparse]).tocsr()
        
        logit_roc_auc = logit_roc_auc_score(X_train_sparse, y_values)
        feature_scores.append((logit_roc_auc, fe_))

sorted_features = sorted(feature_scores, key=lambda x: x[0], reverse=True)

print('5 лучших метрик с фичами')
for pair in sorted_features[:5]:
    print('roc_auc_score:', pair[0], 'features:', pair[1])

print('\n5 худших метрик с фичами')
for pair in sorted_features[-5:]:
    print('roc_auc_score:', pair[0], 'features:', pair[1])

5 лучших метрик с фичами
roc_auc_score: 0.934381940306 features: ['weekend']
roc_auc_score: 0.933847286791 features: []
roc_auc_score: 0.924980978083 features: ['day_of_week', 'weekend']
roc_auc_score: 0.924717063844 features: ['day_of_week']
roc_auc_score: 0.923196690838 features: ['#unique_sites', 'weekend']

5 худших метрик с фичами
roc_auc_score: 0.674627312681 features: ['session_timespan', 'day_of_week', 'weekend']
roc_auc_score: 0.668984875716 features: ['session_timespan', 'weekend']
roc_auc_score: 0.66837137001 features: ['session_timespan', 'time_of_day', 'weekend']
roc_auc_score: 0.666589603811 features: ['session_timespan', 'time_of_day']
roc_auc_score: 0.665770081657 features: ['session_timespan']


**ВЫВОД:**    
Результат немного странный (если, конечно, не закралась системная ошибка), но тем не менее -   
только добавление **одного** из всех признаков - а именно **'weekend'** - дало **увеличение** метрики **ROC AUC**.
Остальные признаки только лишь **уменьшили** метрику **ROC AUC**.   
Причем особенно **негативно** влияет присутствие признака **'session_timespan'**.   
#### ИТАК, добавляем только **'weekend'**.

Добавим  **'weekend'**, подготовим данные и сформируем посылку на **Kaggle**.

In [31]:
%%time
train_sites_sparse, features_df, y_train = feature_engineering_df(train_test_df, features=features, train=True)
features_sparse = csr_matrix(features_df[['weekend']].values)
X_train_test_sparse_newfe = sparse.hstack([train_sites_sparse , features_sparse]).tocsr()

# делим на трейн и тест
X_train_sparse_newfe = X_train_test_sparse_newfe[:train_df.shape[0]]
X_test_sparse_newfe = X_train_test_sparse_newfe[train_df.shape[0]:]
y_train = y_train[:train_df.shape[0]]

Wall time: 6min 43s


In [32]:
%%time
sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17)
logit_test_pred_proba = sgd_logit.fit(X_train_sparse_newfe, y_train).predict_proba(X_test_sparse_newfe)[:, 1]
    
write_to_submission_file(logit_test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_sgd_logit_newfe.csv'))

Wall time: 1.28 s


Сделаем соответствующую **посылку на Kaggle**.   
Её результат:   
- **+46** позиций вверх    
- Score Public Leaderboard = **0.92600**   
- Бенчмарк "sgd_logit_benchmark.csv" на публичной части рейтинга в соревновании Kaggle - **побит**, программа *минимум* - **выполнена**.

## Что далее

**Длиной сессии** и **шириной окна** и  поиграть, я так понимаю, **не получится**, поскольку в **тестовой** выборке **жестко задано** (10, 10).

Поработаем с настройками SGDClassifier:
- альфа
- балансировка классов

Для оптимизации временн**ы**х затрат поработаем с выборкой на 150 пользователей.

Загрузим сохраненные ранее данные и сформируем выборку для работы.

In [33]:
with open(os.path.join(PATH_TO_DATA, 'selected_features_150users.pkl'), 'rb') as selected_features_150users_pkl:
    selected_features_150users = pickle.load(selected_features_150users_pkl)
    
with open(os.path.join(PATH_TO_DATA, 'new_features_150users.pkl'), 'rb') as new_features_150users_pkl:
    new_features_150users = pickle.load(new_features_150users_pkl)
    
with open(os.path.join(PATH_TO_DATA, 'X_sparse_150users.pkl'), 'rb') as X_sparse_150users_pkl:
    X_sparse_150users = pickle.load(X_sparse_150users_pkl)
    
with open(os.path.join(PATH_TO_DATA, 'y_150users.pkl'), 'rb') as y_150users_pkl:
    y_150users = pickle.load(y_150users_pkl)

In [34]:
selected_features_150users.head()

,start_year,start_month,start_day,time_of_day,freq_top30_0_1,freq_top30_1_3,freq_top30_2_41,freq_top30_3_2,freq_top30_4_181,freq_top30_5_214,...,freq_top30_21_1201,freq_top30_22_6,freq_top30_23_2165,freq_top30_24_244,freq_top30_25_180,freq_top30_26_1463,freq_top30_27_106,freq_top30_28_253,freq_top30_29_2707,weekend
0,2013,11,15,1,4,2,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2013,11,15,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,2013,11,15,1,5,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2013,11,15,1,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2013,11,15,1,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
selected_features_150users.shape, y_150users.shape

((137019, 35), (137019,))

In [36]:
X_sparse_150users

<137019x27797 sparse matrix of type '<class 'numpy.int32'>'
	with 1369510 stored elements in Compressed Sparse Row format>

Вырежем нужные новые признаки ('weekend') и добавим к "мешку слов" с сайтами. Скорректируем также целевой признак.

In [37]:
X_150_newfe_sparse = csr_matrix(selected_features_150users[['weekend']].values)
  
X_150_train_newfe_sparse = sparse.hstack([X_sparse_150users, X_150_newfe_sparse]).tocsr()

In [38]:
y_150_train = (y_150users == 128).astype(int)

Для подбора параметров воспльзуемся **GridSearchCV**:

In [39]:
%%time
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)

sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17)

sgd_params = {'alpha': np.logspace(-8, 1, 40)}

sgd_grid_searcher = GridSearchCV(sgd_logit, param_grid=sgd_params, scoring='roc_auc', n_jobs=-1, cv=skf)
sgd_grid_searcher.fit(X_150_train_newfe_sparse, y_150_train)

Wall time: 58.2 s


In [40]:
print(sgd_grid_searcher.best_params_, sgd_grid_searcher.best_score_)

{'alpha': 4.9238826317067415e-05} 0.971957399925


Попробуем то же, но **поменяем балансировку классов** в sgd

In [41]:
%%time
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)

sgd_logit = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)

sgd_params = {'alpha': np.logspace(-8, 1, 40)}

sgd_grid_searcher = GridSearchCV(sgd_logit, param_grid=sgd_params, scoring='roc_auc', n_jobs=-1, cv=skf)
sgd_grid_searcher.fit(X_150_train_newfe_sparse, y_150_train)

Wall time: 38.1 s


In [42]:
print(sgd_grid_searcher.best_params_, sgd_grid_searcher.best_score_)

{'alpha': 0.00041246263829013564} 0.977036274013


ВЫВОД:    
результат **улучшился**.    
Обучим на этих параметрах классификатор и сформируем посылку на **Kaggle**

In [43]:
%%time
# train_sites_sparse, features_df, y_train = feature_engineering_df(train_test_df, features=features, train=True)
# features_sparse = csr_matrix(features_df[['weekend']].values)
# X_train_test_sparse_newfe = sparse.hstack([train_sites_sparse , features_sparse]).tocsr()

# делим на трейн и тест
X_train_sparse_newfe = X_train_test_sparse_newfe[:train_df.shape[0]]
X_test_sparse_newfe = X_train_test_sparse_newfe[train_df.shape[0]:]
y_train = y_train[:train_df.shape[0]]

sgd_logit = SGDClassifier(loss='log', n_jobs=-1, alpha=sgd_grid_searcher.best_params_['alpha'], 
                          class_weight='balanced', random_state=17)
logit_test_pred_proba = sgd_logit.fit(X_train_sparse_newfe, y_train).predict_proba(X_test_sparse_newfe)[:, 1]
    
write_to_submission_file(logit_test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_sgd_logit_newfe_alpha_bal.csv'))

Wall time: 1.34 s


При таких параметрах **на Leader Board** auc_roc больше не стал, наоборот - **уменьшился**: **0.92275** (было 0.92600).    
Будем подбирать дальше.

In [44]:
sgd_logit = SGDClassifier(loss='log', n_jobs=-1, 
                          class_weight='balanced', random_state=17)
logit_test_pred_proba = sgd_logit.fit(X_train_sparse_newfe, y_train).predict_proba(X_test_sparse_newfe)[:, 1]
    
write_to_submission_file(logit_test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_sgd_logit_newfe_bal.csv'))

Score on Leader Board: **0.91911** - **уменьшился**

#### Поработаем еще с фичами.

In [45]:
def CV_score(estimator, X_train_sparse, y, cv=None, random_state=17, test_size=0.3):
    '''
    обучение и расчет метрики на отложенной выборке
    '''
    # Разобьем выборку на 2 части. Учтем, что разбиение выборки с train_test_split должно быть стратифицированным
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_sparse, y, 
                                                          test_size=test_size, 
                                                          random_state=random_state, stratify=y)
    mean_CV_score = cross_val_score(estimator, X_train, y_train, cv=cv, scoring='roc_auc', n_jobs=-1).mean()
 
    return mean_CV_score

In [48]:
%%time
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)
features = ['#unique_sites', 'session_timespan', 'start_hour', 'day_of_week', 'time_of_day', 'weekend']
feature_scores = []
features_150users = pd.concat([new_features_150users, selected_features_150users], axis=1)
for r in range(len(features) + 1):
    for fe in itertools.combinations(features, r):
        fe_ = list(fe)
        X_150_new_sparse = csr_matrix(features_150users[fe_].values)
  
        X_150_train_new_sparse = sparse.hstack([X_sparse_150users, X_150_new_sparse]).tocsr()
        
        estimator = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
        roc_auc_score = CV_score(estimator, X_150_train_new_sparse, y_150_train, cv=skf, 
                                 random_state=17, test_size=0.3)
        feature_scores.append((roc_auc_score, fe_))

sorted_features = sorted(feature_scores, key=lambda x: x[0], reverse=True)

print('5 лучших метрик с фичами')
for pair in sorted_features[:5]:
    print('roc_auc_score:', pair[0], 'features:', pair[1])

print('\n5 худших метрик с фичами')
for pair in sorted_features[-5:]:
    print('roc_auc_score:', pair[0], 'features:', pair[1])

5 лучших метрик с фичами
roc_auc_score: 0.975041438505 features: ['#unique_sites', 'time_of_day', 'weekend']
roc_auc_score: 0.974795606899 features: ['#unique_sites', 'day_of_week', 'time_of_day', 'weekend']
roc_auc_score: 0.974109871176 features: ['day_of_week', 'time_of_day', 'weekend']
roc_auc_score: 0.973203642856 features: ['#unique_sites', 'weekend']
roc_auc_score: 0.972939022961 features: ['day_of_week', 'weekend']
/n5 худших метрик с фичами
roc_auc_score: 0.595232749661 features: ['#unique_sites', 'session_timespan', 'time_of_day', 'weekend']
roc_auc_score: 0.592947564512 features: ['#unique_sites', 'session_timespan', 'time_of_day']
roc_auc_score: 0.59078323166 features: ['#unique_sites', 'session_timespan', 'weekend']
roc_auc_score: 0.588683480401 features: ['#unique_sites', 'session_timespan']
roc_auc_score: 0.515455859856 features: ['session_timespan', 'weekend']
Wall time: 13min 54s


## Далее будем учитывать обстоятельство, что большинство дополнительных фич у нас по сути КАТЕГОРИАЛЬНЫЕ, а учитываются они пока КАК КОЛИЧЕСТВЕННЫЕ (то есть с отношением порядка)

В частности, **становится понятным**, почему ранее "выстрелила" только **одна** из всех фич - **'weekend'**. Эта фича имеет **только два** значения (т.е. *бинарна*), что позволяет корректно учитывать ее в регрессии.   
Для корректности остальные категориальные фичи надо тоже **бинаризовать**. Более правильно - это сделать **'one hot coding'**, или (что пока проще и позволяет природа этих признаков) - **бинаризовать по некоему порогу**.

**Порог** можно определить "на глаз", или взять как среднее арифметическое (для бинарной классификации) медиан при группировке по целевому признаку

В соответствии с этими соображениями модифицируем функцию для подготовки признаков для бинаризации **по порогу**.

In [49]:
def feature_engineering_df_bin(train_df, features=None, train=True):
    '''
    features = ['#unique_sites_bin', 'session_timespan_bin', 'time_of_day', 'weekend']
    '''
    
    if features == None:
        features_ = ['#unique_sites_bin', 'session_timespan_bin', 'time_of_day', 'weekend']
    else:
        features_ = features
    
    # готовим сайты
    train_df_sites = train_df[['site%d' % i for i in range(1, 11)]] # наны заполним нулями потом
    
    # подготовим временнЫе метки
    train_df[['time%d' % i for i in range(1, 11)]] = train_df[['time%d' % i for i in range(1, 11)]].applymap(pd.to_datetime)
    timestamp_df = train_df[['time%d' % i for i in range(1, 11)]].applymap(lambda x: pd.datetime.timestamp(x) 
                                                                  if type(x) == pd._libs.tslib.Timestamp else np.nan)
    # добавляем признаки
    if '#unique_sites_bin' in features_:
        train_df['#unique_sites_bin'] = (train_df[['site' + str(i) for i in range(1, 11)]].nunique(axis=1, dropna=True)). \
                                         apply(lambda x: 1 if x < 6 else 0)
    if 'session_timespan_bin' in features_:
        train_df['session_timespan_bin'] = (timestamp_df.max(axis=1, skipna=True) - timestamp_df.min(axis=1, skipna=True)). \
                                            apply(lambda x: 1 if x < 50 else 0)
#     if 'start_hour' in features:    
#         train_df['start_hour'] = train_df['time1'].apply(lambda x: x.hour)
#     if 'day_of_week' in features:    
#         train_df['day_of_week'] = train_df['time1'].apply(lambda x: x.weekday())
    if 'time_of_day' in features_:    
        # до обеда - после обеда
        train_df['time_of_day'] = train_df['time1'].apply(lambda x:  1 if x.hour <  14 else 0)
    if 'weekend' in features_:
        # индикатор уикэнда в день начала сессии
        train_df['weekend'] = train_df['time1'].apply(lambda x: 1 if x.weekday() in [5, 6] else 0)
 
    train_df_sites = train_df_sites.fillna(0).astype('int')
    train_sparse = csr_matrix((np.ones(train_df_sites.values.size), 
                                train_df_sites.values.ravel(), 
                                np.arange(train_df_sites.values.shape[0] + 1) * 
                                train_df_sites.values.shape[1]), dtype=int)[:, 1:]
    if train:
        y = train_df['target'].values
    else:
        y = None

    return train_sparse, train_df[features_], y

In [50]:
%%time
train_sites_sparse, selected_features_df, y_values = feature_engineering_df_bin(train_df, train=True)

Wall time: 1min 22s


In [52]:
%%time
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)
features = selected_features_df.columns
feature_scores = []
for r in range(len(features) + 1):
    for fe in itertools.combinations(features, r):
        fe_ = list(fe)
        selected_features_sparse = csr_matrix(selected_features_df[fe_].values)
  
        X_train_new_sparse = sparse.hstack([train_sites_sparse , selected_features_sparse]).tocsr()
        
        estimator = SGDClassifier(loss='log', n_jobs=-1, random_state=17)
        roc_auc_score = CV_score(estimator, X_train_new_sparse, y_values, cv=skf, 
                                 random_state=17, test_size=0.3)
        feature_scores.append((roc_auc_score, fe_))

sorted_features = sorted(feature_scores, key=lambda x: x[0], reverse=True)

print('Без баланса')
print('\n5 лучших метрик с фичами')
for pair in sorted_features[:5]:
    print('roc_auc_score:', pair[0], 'features:', pair[1])

print('\n5 худших метрик с фичами')
for pair in sorted_features[-5:]:
    print('roc_auc_score:', pair[0], 'features:', pair[1])

Без баланса

5 лучших метрик с фичами
roc_auc_score: 0.934218622552 features: ['weekend']
roc_auc_score: 0.933635284594 features: ['time_of_day', 'weekend']
roc_auc_score: 0.932279637877 features: []
roc_auc_score: 0.930773100919 features: ['time_of_day']
roc_auc_score: 0.929718057263 features: ['#unique_sites_bin', 'time_of_day', 'weekend']

5 худших метрик с фичами
roc_auc_score: 0.923128645975 features: ['session_timespan_bin', 'weekend']
roc_auc_score: 0.920592024569 features: ['#unique_sites_bin', 'session_timespan_bin', 'time_of_day']
roc_auc_score: 0.920483235092 features: ['session_timespan_bin']
roc_auc_score: 0.915035943593 features: ['#unique_sites_bin', 'session_timespan_bin', 'weekend']
roc_auc_score: 0.910999149669 features: ['#unique_sites_bin', 'session_timespan_bin']
Wall time: 3min 45s


#### Проделаем то же самое, делая баланс в обучающей выборке

In [53]:
%%time
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)
features = selected_features_df.columns
feature_scores = []
for r in range(len(features) + 1):
    for fe in itertools.combinations(features, r):
        fe_ = list(fe)
        selected_features_sparse = csr_matrix(selected_features_df[fe_].values)
  
        X_train_new_sparse = sparse.hstack([train_sites_sparse , selected_features_sparse]).tocsr()
        
        estimator = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
        roc_auc_score = CV_score(estimator, X_train_new_sparse, y_values, cv=skf, 
                                 random_state=17, test_size=0.3)
        feature_scores.append((roc_auc_score, fe_))

sorted_features = sorted(feature_scores, key=lambda x: x[0], reverse=True)

print('С балансом')
print('\n5 лучших метрик с фичами')
for pair in sorted_features[:5]:
    print('roc_auc_score:', pair[0], 'features:', pair[1])

print('\n5 худших метрик с фичами')
for pair in sorted_features[-5:]:
    print('roc_auc_score:', pair[0], 'features:', pair[1])

С балансом

5 лучших метрик с фичами
roc_auc_score: 0.938555892249 features: ['#unique_sites_bin', 'session_timespan_bin', 'time_of_day', 'weekend']
roc_auc_score: 0.936702678996 features: ['#unique_sites_bin', 'session_timespan_bin', 'time_of_day']
roc_auc_score: 0.936023752468 features: ['#unique_sites_bin', 'session_timespan_bin', 'weekend']
roc_auc_score: 0.935668706936 features: ['session_timespan_bin', 'time_of_day', 'weekend']
roc_auc_score: 0.934530145929 features: ['#unique_sites_bin', 'session_timespan_bin']

5 худших метрик с фичами
roc_auc_score: 0.930224205183 features: ['weekend']
roc_auc_score: 0.930118730518 features: ['session_timespan_bin']
roc_auc_score: 0.929995398298 features: ['#unique_sites_bin']
roc_auc_score: 0.929944599468 features: ['time_of_day']
roc_auc_score: 0.926259578693 features: []
Wall time: 3min 44s


Возьмем комбинацию лучших фич и на ее основе сформируем посылку на **Kaggle**.

In [54]:
sorted_features[0][1]

['#unique_sites_bin', 'session_timespan_bin', 'time_of_day', 'weekend']

In [55]:
%%time
features = ['#unique_sites_bin', 'session_timespan_bin', 'time_of_day', 'weekend']
train_sites_sparse, selected_features_df, y_values
X_train_test_sites_sparse, selected_train_test_features_df, y_train_none = \
                                                        feature_engineering_df_bin(train_test_df, features=features, train=True)
# делим на трейн и тест
selected_train_test_features_sparse = csr_matrix(selected_train_test_features_df[features].values)
  
X_train_test_newfe_sparse = sparse.hstack([X_train_test_sites_sparse , selected_train_test_features_sparse]).tocsr()
X_train_newfe_sparse = X_train_test_newfe_sparse[:train_df.shape[0]]
X_test_newfe_sparse = X_train_test_newfe_sparse[train_df.shape[0]:]
y_train = y_train_none[:train_df.shape[0]]

sgd_logit = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
logit_test_pred_proba = sgd_logit.fit(X_train_newfe_sparse, y_train).predict_proba(X_test_newfe_sparse)[:, 1]
    
write_to_submission_file(logit_test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_sgd_logit_4features_bal.csv'))

Wall time: 1min 50s


Результат "так себе" - **0.90482**

In [56]:
%%time
features = ['time_of_day', 'weekend']
train_sites_sparse, selected_features_df, y_values
X_train_test_sites_sparse, selected_train_test_features_df, y_train_none = \
                                                        feature_engineering_df_bin(train_test_df, features=features, train=True)
# делим на трейн и тест
selected_train_test_features_sparse = csr_matrix(selected_train_test_features_df[features].values)
  
X_train_test_newfe_sparse = sparse.hstack([X_train_test_sites_sparse , selected_train_test_features_sparse]).tocsr()
X_train_newfe_sparse = X_train_test_newfe_sparse[:train_df.shape[0]]
X_test_newfe_sparse = X_train_test_newfe_sparse[train_df.shape[0]:]
y_train = y_train_none[:train_df.shape[0]]

sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17)
logit_test_pred_proba = sgd_logit.fit(X_train_newfe_sparse, y_train).predict_proba(X_test_newfe_sparse)[:, 1]
    
write_to_submission_file(logit_test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_sgd_logit_2features.csv'))

Wall time: 1min 13s


Еще хуже - **0.87852**

In [57]:
%%time
# готовим признаки
features = ['#unique_sites_bin', 'session_timespan_bin', 'time_of_day', 'weekend']
train_sites_sparse, selected_features_df, y_values
X_train_test_sites_sparse, selected_train_test_features_df, y_train_none = \
                                                        feature_engineering_df_bin(train_test_df, features=features, train=True)
###### делим на трейн и тест
selected_train_test_features_sparse = csr_matrix(selected_train_test_features_df[features].values)
  
X_train_test_newfe_sparse = sparse.hstack([X_train_test_sites_sparse , selected_train_test_features_sparse]).tocsr()
X_train_newfe_sparse = X_train_test_newfe_sparse[:train_df.shape[0]]
X_test_newfe_sparse = X_train_test_newfe_sparse[train_df.shape[0]:]
y_train = y_train_none[:train_df.shape[0]]

# подбор альфа
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)

sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17)

sgd_params = {'alpha': np.logspace(-8, 1, 40)}

sgd_grid_searcher = GridSearchCV(sgd_logit, param_grid=sgd_params, scoring='roc_auc', n_jobs=-1, cv=skf)
sgd_grid_searcher.fit(X_train_newfe_sparse, y_train)

print('best:','\nparams =', sgd_grid_searcher.best_params_, '  roc_auc =', sgd_grid_searcher.best_score_)

best: 
params = {'alpha': 1.7012542798525893e-05}   roc_auc = 0.951589553787
Wall time: 2min 38s


In [58]:
%%time
# подбор альфа (balanced)
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)

sgd_logit = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)

sgd_params = {'alpha': np.logspace(-8, 1, 40)}

sgd_grid_searcher = GridSearchCV(sgd_logit, param_grid=sgd_params, scoring='roc_auc', n_jobs=-1, cv=skf)
sgd_grid_searcher.fit(X_train_newfe_sparse, y_train)

print('best:','\nparams =', sgd_grid_searcher.best_params_, '  roc_auc =', sgd_grid_searcher.best_score_)

best: 
params = {'alpha': 0.00041246263829013564}   roc_auc = 0.952625453824
Wall time: 49.6 s


In [59]:
%%time
# features = ['#unique_sites_bin', 'session_timespan_bin', 'time_of_day', 'weekend']
# train_sites_sparse, selected_features_df, y_values
# X_train_test_sites_sparse, selected_train_test_features_df, y_train_none = \
#                                                         feature_engineering_df_bin(train_test_df, features=features, train=True)
# # делим на трейн и тест
# selected_train_test_features_sparse = csr_matrix(selected_train_test_features_df[features].values)
  
# X_train_test_newfe_sparse = sparse.hstack([X_train_test_sites_sparse , selected_train_test_features_sparse]).tocsr()
# X_train_newfe_sparse = X_train_test_newfe_sparse[:train_df.shape[0]]
# X_test_newfe_sparse = X_train_test_newfe_sparse[train_df.shape[0]:]
# y_train = y_train_none[:train_df.shape[0]]

sgd_logit = SGDClassifier(loss='log', n_jobs=-1, alpha=sgd_grid_searcher.best_params_['alpha'], 
                          class_weight='balanced', random_state=17)
logit_test_pred_proba = sgd_logit.fit(X_train_newfe_sparse, y_train).predict_proba(X_test_newfe_sparse)[:, 1]
    
write_to_submission_file(logit_test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_sgd_logit_4features_bal_alpha.csv'))

Wall time: 1.31 s


**Kaggle: 0.90482**

### пробуем CatBoostClassifier
```python
%%time
# пробуем CatBoostClassifier
import catboost
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)
features = ['time_of_day', 'weekend']
feature_scores = []
for r in range(len(features) + 1):
    for fe in itertools.combinations(features, r):
        fe_ = list(fe)
        selected_features_sparse = csr_matrix(selected_features_df[fe_].values)
  
        X_train_new_sparse = sparse.hstack([train_sites_sparse , selected_features_sparse]).tocsr()
        
        estimator = catboost.CatBoostClassifier()
        roc_auc_score = CV_score(estimator, X_train_new_sparse.toarray(), y_values, cv=skf, 
                                 score='roc_auc', random_state=17, test_size=0.3)
        feature_scores.append((roc_auc_score, fe_))

sorted_features = sorted(feature_scores, key=lambda x: x[0], reverse=True)

# print('С балансом')
# print('\n5 лучших метрик с фичами')
# for pair in sorted_features[:5]:
#     print('roc_auc_score:', pair[0], 'features:', pair[1])

# print('\n5 худших метрик с фичами')
# for pair in sorted_features[-5:]:
#     print('roc_auc_score:', pair[0], 'features:', pair[1])
for pair in sorted_features:
    print('roc_auc_score:', pair[0], 'features:', pair[1])
    
```
**РЕЗЮМЕ:**    
Catboost **не поддерживае**т работу с **sparse**-форматом, поэтому пока не подходит для этой задачи (а перевод в np.array приводит к **ошибке памяти**, скорее всего - памяти для этого просто не хватает)

### Пробуем XGBOOST, поскольку он умеет работать со sparse-форматом

In [60]:
import xgboost as xgb

In [61]:
%%time
# пробуем xgboost
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)
features = ['time_of_day', 'weekend']
feature_scores = []
for r in range(len(features) + 1):
    for fe in itertools.combinations(features, r):
        fe_ = list(fe)
        selected_features_sparse = csr_matrix(selected_features_df[fe_].values)
  
        X_train_new_sparse = sparse.hstack([train_sites_sparse , selected_features_sparse]).tocsr()
        
        estimator = xgb.XGBClassifier(n_jobs=-1, random_state=17)
        roc_auc_score = CV_score(estimator, X_train_new_sparse, y_values, cv=skf, 
                                 random_state=17, test_size=0.3)
        feature_scores.append((roc_auc_score, fe_))

sorted_features = sorted(feature_scores, key=lambda x: x[0], reverse=True)

# print('С балансом')
# print('\n5 лучших метрик с фичами')
# for pair in sorted_features[:5]:
#     print('roc_auc_score:', pair[0], 'features:', pair[1])

# print('\n5 худших метрик с фичами')
# for pair in sorted_features[-5:]:
#     print('roc_auc_score:', pair[0], 'features:', pair[1])
for pair in sorted_features:
    print('roc_auc_score:', pair[0], 'features:', pair[1])

roc_auc_score: 0.904199430799 features: ['time_of_day', 'weekend']
roc_auc_score: 0.899112212739 features: ['time_of_day']
roc_auc_score: 0.890882452164 features: ['weekend']
roc_auc_score: 0.884568244521 features: []
Wall time: 2min 50s


Модифицируем **feature_engineering_df**, "причесав" **session_timespan**:

In [62]:
def feature_engineering_df_mod(train_df, test_df=None, features=None):
    '''
    features = ['#unique_sites', 'session_timespan', 'start_hour', 'day_of_week', 'time_of_day', 'weekend']
    '''
    if features is None:
        features = ['#unique_sites', 'session_timespan', 'start_hour', 'day_of_week', 'time_of_day', 'weekend']
    
    if test_df is None:
        df = train_df
    else:
        df = pd.concat([train_df, test_df])
    
    # выделим отдельно сайты
    df_sites = df[['site%d' % i for i in range(1, 11)]] # не забыть заполнить нули
    
    # подготовим временнЫе метки
    df[['time%d' % i for i in range(1, 11)]] = df[['time%d' % i for i in range(1, 11)]].applymap(pd.to_datetime)
    timestamp_df = df[['time%d' % i for i in range(1, 11)]].applymap(lambda x: pd.datetime.timestamp(x) 
                                                                  if type(x) == pd._libs.tslib.Timestamp else np.nan)
    # добавляем признаки
    if '#unique_sites' in features:
        df['#unique_sites'] = df[['site' + str(i) for i in range(1, 11)]].nunique(axis=1, dropna=True)
    if 'session_timespan' in features:
        df['session_timespan'] = timestamp_df.max(axis=1, skipna=True) - timestamp_df.min(axis=1, skipna=True)
        df['session_timespan'] = df['session_timespan'].apply(lambda x: x // 30 if x < 210 else 7)
    if 'start_hour' in features:    
        df['start_hour'] = df['time1'].apply(lambda x: x.hour)
    if 'day_of_week' in features:    
        df['day_of_week'] = df['time1'].apply(lambda x: x.weekday())
    if 'time_of_day' in features:    
        # время суток (06-12 - утро(1), 12-19 - день(2), 19-24 - вечер(3), 00-06 - ночь(4))
        df['time_of_day'] = df['time1'].apply(lambda x:  1 if x.hour >  6 and x.hour <= 12 else 
                                                                     2 if x.hour > 12 and x.hour <= 19 else
                                                                     3 if x.hour > 19 and x.hour <= 24 else 
                                                                     4)
    if 'weekend' in features:
        # индикатор уикэнда в день начала сессии
        df['weekend'] = df['time1'].apply(lambda x: 1 if x.weekday() in [5, 6] else 0)
    
    X_new_sparse = csr_matrix(df[features].values)
    
    df_sites = df_sites.fillna(0).astype('int')
    train_sparse = csr_matrix((np.ones(df_sites.values.size), 
                                df_sites.values.ravel(), 
                                np.arange(df_sites.values.shape[0] + 1) * 
                                df_sites.values.shape[1]), dtype=int)[:, 1:]
    
    
    y_train = df['target'].values[:train_df.shape[0]]
        
    X_sparse = sparse.hstack([train_sparse, X_new_sparse]).tocsr()
    X_train_sparse = X_sparse[:train_df.shape[0]]
    X_test_sparse = X_sparse[train_df.shape[0]:]
    
    
    
    return (X_train_sparse, y_train) if test_df is None else (X_train_sparse, y_train, X_test_sparse)

In [63]:
%%time
# готовим данные
X_train_new_sparse, y_train = feature_engineering_df_mod(train_df)

Wall time: 1min 25s


In [64]:
# инициализируем параметры
params = {
    'objective': 'binary:logistic',
    'max_depth': 3,
    'learning_rate': 0.5,
    'silent': 1.0,
    'n_estimators': 170
}

In [65]:
estimator = xgb.XGBClassifier(n_jobs=-1, random_state=17, **params)
roc_auc_score = CV_score(estimator, X_train_new_sparse, y_train, 
                         random_state=17, test_size=0.3)
roc_auc_score

0.98481937043309076

**roc_auc** получился довольно приличный, подготовим и сделаем на этих параметрах посылку на **Kaggle**

In [66]:
%%time
X_train_sparse, y_train, X_test_sparse = feature_engineering_df_mod(train_df, test_df)

Wall time: 2min 52s


In [67]:
%%time
params = {
    'objective': 'binary:logistic',
    'max_depth': 3,
    'learning_rate': 0.5,
    'silent': 1.0,
    'n_estimators': 170
}


estimator = xgb.XGBClassifier(n_jobs=-1, random_state=17, **params)
test_pred_proba = estimator.fit(X_train_sparse, y_train).predict_proba(X_test_sparse)[:, 1]
    
write_to_submission_file(test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_xgb_allfeatures_1.csv'))

Wall time: 23.8 s


**Kaggle: 0.90088**     
:(

In [68]:
%%time
params = {
    'objective': 'binary:logistic',
    'max_depth': 3,
    'learning_rate': 0.5,
    'silent': 1.0,
    'n_estimators': 1000
}


estimator = xgb.XGBClassifier(n_jobs=-1, random_state=17, **params)
test_pred_proba = estimator.fit(X_train_sparse, y_train).predict_proba(X_test_sparse)[:, 1]
    
write_to_submission_file(test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_xgb_allfeatures_n_est1000.csv'))

Wall time: 2min 11s


**Kaggle: 0.85717**     
:(((

In [69]:
%%time
# инициализируем параметры
params = {
    'objective': 'binary:logistic',
    'max_depth': 3,
    'learning_rate': 0.5,
    'silent': 1.0,
    'n_estimators': 170,
    'eval_metric': 'auc'
}

estimator = xgb.XGBClassifier(n_jobs=-1, cv=skf, random_state=17, **params)
roc_auc_score = CV_score(estimator, X_train_new_sparse, y_train, 
                         random_state=17, test_size=0.3)

print(roc_auc_score)

0.984819370433
Wall time: 49.3 s


**Промежуточное РЕЗЮМЕ:**   
Видим, что **xgboost с логисической регрессией**, несмотря на **высокие результаты на кроссвалидации**, на **Kaggle** хорошего результата **не дает**. Поэтому пока откажемся от него. (Теоретически его можно было бы поднастроить и использовать **для смешивания** (блендинга) ответов бустинга и линейной модели.)

## Вернемся к **SGDClassifier**
пропустим категориальные признаки через **OHE** и "скормим" SGDClassifier, где вначале получился высокий AUC

In [70]:
from sklearn.preprocessing import OneHotEncoder

Модифицируем подготовку фич, добавив OHE

In [71]:
def feature_engineering_df_OHE(train_df, test_df=None, features=None):
    '''
    features = ['#unique_sites', 'session_timespan', 'start_hour', 'day_of_week', 'time_of_day', 'weekend']
    '''
    if features is None:
        features = ['#unique_sites', 'session_timespan', 'start_hour', 'day_of_week', 'time_of_day', 'weekend']
    
    if test_df is None:
        df = train_df
    else:
        df = pd.concat([train_df, test_df])
    
    # выделим отдельно сайты
    df_sites = df[['site%d' % i for i in range(1, 11)]] # не забыть заполнить нули
    
    # подготовим временнЫе метки
    df[['time%d' % i for i in range(1, 11)]] = df[['time%d' % i for i in range(1, 11)]].applymap(pd.to_datetime)
    timestamp_df = df[['time%d' % i for i in range(1, 11)]].applymap(lambda x: pd.datetime.timestamp(x) 
                                                                  if type(x) == pd._libs.tslib.Timestamp else np.nan)
    # добавляем признаки
    if '#unique_sites' in features:
        df['#unique_sites'] = df[['site' + str(i) for i in range(1, 11)]].nunique(axis=1, dropna=True)
    if 'session_timespan' in features:
        df['session_timespan'] = timestamp_df.max(axis=1, skipna=True) - timestamp_df.min(axis=1, skipna=True)
        df['session_timespan'] = df['session_timespan'].apply(lambda x: x // 30 if x < 210 else 7)
    if 'start_hour' in features:    
        df['start_hour'] = df['time1'].apply(lambda x: x.hour)
    if 'day_of_week' in features:    
        df['day_of_week'] = df['time1'].apply(lambda x: x.weekday())
    if 'time_of_day' in features:    
        # время суток (06-12 - утро(1), 12-19 - день(2), 19-24 - вечер(3), 00-06 - ночь(4))
        df['time_of_day'] = df['time1'].apply(lambda x:  1 if x.hour >  6 and x.hour <= 12 else 
                                                                     2 if x.hour > 12 and x.hour <= 19 else
                                                                     3 if x.hour > 19 and x.hour <= 24 else 
                                                                     4)
    if 'weekend' in features:
        # индикатор уикэнда в день начала сессии
        df['weekend'] = df['time1'].apply(lambda x: 1 if x.weekday() in [5, 6] else 0)
    
    encoder = OneHotEncoder()
    #X_new_sparse = csr_matrix(df[features].values)
    X_new_sparse = encoder.fit_transform(df[features].values)
    
    df_sites = df_sites.fillna(0).astype('int')
    train_sparse = csr_matrix((np.ones(df_sites.values.size), 
                                df_sites.values.ravel(), 
                                np.arange(df_sites.values.shape[0] + 1) * 
                                df_sites.values.shape[1]), dtype=int)[:, 1:]
    
    
    y_train = df['target'].values[:train_df.shape[0]]
        
    X_sparse = sparse.hstack([train_sparse, X_new_sparse]).tocsr()
    X_train_sparse = X_sparse[:train_df.shape[0]]
    X_test_sparse = X_sparse[train_df.shape[0]:]
    
    
    
    return (X_train_sparse, y_train) if test_df is None else (X_train_sparse, y_train, X_test_sparse)

In [72]:
%%time
X_train_sparse_ohe, y_train_ohe, X_test_sparse_ohe = feature_engineering_df_OHE(train_df, test_df)

Wall time: 2min 52s


In [73]:
%%time
estimator = SGDClassifier(loss='log', n_jobs=-1, random_state=17)
auc = CV_score(estimator, X_train_sparse_ohe, y_train_ohe, cv=skf)
print(auc)

0.974101262722
Wall time: 19.3 s


Результат впечатляет, попробуем на **Kaggle**

In [74]:
%%time
estimator = SGDClassifier(loss='log', n_jobs=-1, random_state=17)
test_pred_proba = estimator.fit(X_train_sparse_ohe, y_train_ohe).predict_proba(X_test_sparse_ohe)[:, 1]
    
write_to_submission_file(test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_sgd_allfeatures_OHE.csv'))

Wall time: 1.28 s


**Kaggle: 0.93742**    
Наконец то!   
:))

Попробуем с балансировкой классов

In [75]:
%%time
estimator = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
auc = CV_score(estimator, X_train_sparse_ohe, y_train_ohe, cv=skf)
print(auc)

0.977356917229
Wall time: 14.5 s


Результат улучшился. Сделаем посылку на Kaggle

In [76]:
%%time
estimator = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
test_pred_proba = estimator.fit(X_train_sparse_ohe, y_train_ohe).predict_proba(X_test_sparse_ohe)[:, 1]
    
write_to_submission_file(test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_sgd_allfeatures_OHE_bal.csv'))

Wall time: 1.35 s


**Kaggle: 0.94607** - результат **еще более улучшился!**   
:)))

Сделаем подбор **alpha**

In [77]:
%%time
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)

sgd_logit = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)

sgd_params = {'alpha': np.logspace(-8, 1, 40)}

sgd_grid_searcher = GridSearchCV(sgd_logit, param_grid=sgd_params, scoring='roc_auc', n_jobs=-1, cv=skf)
sgd_grid_searcher.fit(X_train_sparse_ohe, y_train_ohe)

print(sgd_grid_searcher.best_params_, sgd_grid_searcher.best_score_)

{'alpha': 0.00041246263829013564} 0.982048226764
Wall time: 55.5 s


In [78]:
%%time
estimator = SGDClassifier(loss='log', n_jobs=-1, alpha=sgd_grid_searcher.best_params_['alpha'], 
                          class_weight='balanced', random_state=17)
test_pred_proba = estimator.fit(X_train_sparse_ohe, y_train_ohe).predict_proba(X_test_sparse_ohe)[:, 1]
    
write_to_submission_file(test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_sgd_allfeatures_OHE_bal_alpha.csv'))

Wall time: 1.34 s


**Kaggle: 0.94478 - немного уменьшился**.   

Видим, что подбор **alpha** пока результата на **Kaggle** не дал: несмотря на **увеличение метрики на кроссвалидации**, на Leader Board **результат ухудшился**.  

Попробуем еще одну вещь. Попробуем еще немного **модифицировать признак 'session_timespan'**, а именно: прологарифмируем его по основанию **2** и отбросим дробную часть.

In [79]:
def feature_engineering_df_OHE_log2(train_df, test_df=None, features=None):
    '''
    features = ['#unique_sites', 'session_timespan', 'start_hour', 'day_of_week', 'time_of_day', 'weekend']
    '''
    if features is None:
        features = ['#unique_sites', 'session_timespan', 'start_hour', 'day_of_week', 'time_of_day', 'weekend']
    
    if test_df is None:
        df = train_df
    else:
        df = pd.concat([train_df, test_df])
    
    # выделим отдельно сайты
    df_sites = df[['site%d' % i for i in range(1, 11)]] # не забыть заполнить нули
    
    # подготовим временнЫе метки
    df[['time%d' % i for i in range(1, 11)]] = df[['time%d' % i for i in range(1, 11)]].applymap(pd.to_datetime)
    timestamp_df = df[['time%d' % i for i in range(1, 11)]].applymap(lambda x: pd.datetime.timestamp(x) 
                                                                  if type(x) == pd._libs.tslib.Timestamp else np.nan)
    # добавляем признаки
    if '#unique_sites' in features:
        df['#unique_sites'] = df[['site' + str(i) for i in range(1, 11)]].nunique(axis=1, dropna=True)
    if 'session_timespan' in features:
        df['session_timespan'] = timestamp_df.max(axis=1, skipna=True) - timestamp_df.min(axis=1, skipna=True)
#         df['session_timespan'] = df['session_timespan'].apply(lambda x: x // 30 if x < 210 else 7)
        df['session_timespan'] = df['session_timespan'].apply(lambda x: int(np.log2(x)) if x > 1 else 0)
    if 'start_hour' in features:    
        df['start_hour'] = df['time1'].apply(lambda x: x.hour)
    if 'day_of_week' in features:    
        df['day_of_week'] = df['time1'].apply(lambda x: x.weekday())
    if 'time_of_day' in features:    
        # время суток (06-12 - утро(1), 12-19 - день(2), 19-24 - вечер(3), 00-06 - ночь(4))
        df['time_of_day'] = df['time1'].apply(lambda x:  1 if x.hour >  6 and x.hour <= 12 else 
                                                                     2 if x.hour > 12 and x.hour <= 19 else
                                                                     3 if x.hour > 19 and x.hour <= 24 else 
                                                                     4)
    if 'weekend' in features:
        # индикатор уикэнда в день начала сессии
        df['weekend'] = df['time1'].apply(lambda x: 1 if x.weekday() in [5, 6] else 0)
    
    encoder = OneHotEncoder()
    #X_new_sparse = csr_matrix(df[features].values)
    X_new_sparse = encoder.fit_transform(df[features].values)
    
    df_sites = df_sites.fillna(0).astype('int')
    train_sparse = csr_matrix((np.ones(df_sites.values.size), 
                                df_sites.values.ravel(), 
                                np.arange(df_sites.values.shape[0] + 1) * 
                                df_sites.values.shape[1]), dtype=int)[:, 1:]
    
    
    y_train = df['target'].values[:train_df.shape[0]]
        
    X_sparse = sparse.hstack([train_sparse, X_new_sparse]).tocsr()
    X_train_sparse = X_sparse[:train_df.shape[0]]
    X_test_sparse = X_sparse[train_df.shape[0]:]
    
    
    
    return (X_train_sparse, y_train) if test_df is None else (X_train_sparse, y_train, X_test_sparse)

**Проверим** гипотезу на ранее подготовленной функции с кроссвалидацией.

In [80]:
%%time
X_train_sparse_ohe_l2, y_train_ohe, X_test_sparse_ohe_l2 = feature_engineering_df_OHE_log2(train_df, test_df)
estimator = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
auc = CV_score(estimator, X_train_sparse_ohe_l2, y_train_ohe, cv=skf)
print(auc)

0.976601765937
Wall time: 3min 10s


Видим, что **auc** немного увеличилась (в 4-м знаке, было 0.977325731478). Лимит посылок на **Kaggle** позволяет, поэтому проверим на Leader Board.

In [81]:
%%time
estimator = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
test_pred_proba = estimator.fit(X_train_sparse_ohe_l2, y_train_ohe).predict_proba(X_test_sparse_ohe_l2)[:, 1]
    
write_to_submission_file(test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_sgd_allfeatures_OHE_bal_log2.csv'))

Wall time: 1.33 s


На **Kaggle** - 0.94549. Довольно высокий, но все-таки **немного ниже максимального**.

Далее, можно еще попробовать 
- **TfidfVectorizer()**
- подбор параметра **alpha** более продвинутым методом, например - с помощью библиотеки **hyperopt** (хотя на это я мало надеюсь, поскольку ранее вариант с более высоким **AUC** со CV давал **AUC** на Kaggle меньше)

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [83]:
def feature_engineering_df_OHE_Tfidf(train_df, test_df=None, features=None):
    '''
    features = ['#unique_sites', 'session_timespan', 'start_hour', 'day_of_week', 'time_of_day', 'weekend']
    '''
    if features is None:
        features = ['#unique_sites', 'session_timespan', 'start_hour', 'day_of_week', 'time_of_day', 'weekend']
    
    if test_df is None:
        df = train_df
    else:
        df = pd.concat([train_df, test_df])
    
    # выделим отдельно сайты
    df_sites = df[['site%d' % i for i in range(1, 11)]] # не забыть заполнить нули
    
    # подготовим временнЫе метки
    df[['time%d' % i for i in range(1, 11)]] = df[['time%d' % i for i in range(1, 11)]].applymap(pd.to_datetime)
    timestamp_df = df[['time%d' % i for i in range(1, 11)]].applymap(lambda x: pd.datetime.timestamp(x) 
                                                                  if type(x) == pd._libs.tslib.Timestamp else np.nan)
    # добавляем признаки
    if '#unique_sites' in features:
        df['#unique_sites'] = df[['site' + str(i) for i in range(1, 11)]].nunique(axis=1, dropna=True)
    if 'session_timespan' in features:
        df['session_timespan'] = timestamp_df.max(axis=1, skipna=True) - timestamp_df.min(axis=1, skipna=True)
        df['session_timespan'] = df['session_timespan'].apply(lambda x: x // 30 if x < 210 else 7)
#         df['session_timespan'] = df['session_timespan'].apply(lambda x: int(np.log2(x)) if x > 1 else 0)
    if 'start_hour' in features:    
        df['start_hour'] = df['time1'].apply(lambda x: x.hour)
    if 'day_of_week' in features:    
        df['day_of_week'] = df['time1'].apply(lambda x: x.weekday())
    if 'time_of_day' in features:    
        # время суток (06-12 - утро(1), 12-19 - день(2), 19-24 - вечер(3), 00-06 - ночь(4))
        df['time_of_day'] = df['time1'].apply(lambda x:  1 if x.hour >  6 and x.hour <= 12 else 
                                                                     2 if x.hour > 12 and x.hour <= 19 else
                                                                     3 if x.hour > 19 and x.hour <= 24 else 
                                                                     4)
    if 'weekend' in features:
        # индикатор уикэнда в день начала сессии
        df['weekend'] = df['time1'].apply(lambda x: 1 if x.weekday() in [5, 6] else 0)
        
#     X_new_sparse = csr_matrix(df[features].values)
    encoder = OneHotEncoder()
    X_new_sparse = encoder.fit_transform(df[features].values)
    
    df_sites = df_sites.fillna(0).astype(int)
#     train_sparse = csr_matrix((np.ones(df_sites.values.size), 
#                                 df_sites.values.ravel(), 
#                                 np.arange(df_sites.values.shape[0] + 1) * 
#                                 df_sites.values.shape[1]), dtype=int)[:, 1:]

    # готовим блок с сайтами для кодирования посредством TfidfVectorizer()
    df_sites = df_sites.astype(str)
    sites_text = df_sites['site1']
    for i in range(2, 11):
        sites_text += ' ' + df_sites['site{}'.format(i)]
    # кодируем подготовленный блок с сайтами
    Tfidf_Vectorizer = TfidfVectorizer()    
    train_sparse = Tfidf_Vectorizer.fit_transform(sites_text.values)
   
    X_sparse = sparse.hstack([train_sparse, X_new_sparse]).tocsr()
    X_train_sparse = X_sparse[:train_df.shape[0]]
    X_test_sparse = X_sparse[train_df.shape[0]:]
    
    y_train = df['target'].values[:train_df.shape[0]]
    
    return (X_train_sparse, y_train) if test_df is None else (X_train_sparse, y_train, X_test_sparse)

In [84]:
%%time
X_train_sparse_ohe_tfidf, y_train_ohe, X_test_sparse_ohe_tfidf = feature_engineering_df_OHE_Tfidf(train_df, test_df)
estimator = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
auc = CV_score(estimator, X_train_sparse_ohe_tfidf, y_train_ohe, cv=skf)
print(auc)

0.978230775915
Wall time: 3min 23s


Особого выигрыша не дало, но тем не менее попробуем на **Kaggle**:

In [85]:
%%time
estimator = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
test_pred_proba = estimator.fit(X_train_sparse_ohe_tfidf, y_train_ohe).predict_proba(X_test_sparse_ohe_tfidf)[:, 1]
    
write_to_submission_file(test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_sgd_allfeatures_OHE_bal_tfidf.csv'))

Wall time: 1.39 s


**Kaggle: 0.94773**   
Урра!!! Позицию еще хоть и немного, но все же улучшили!   
:)))   

## Попробуем смешивание алгоритмов
Будем делать это по схеме **блендинга**, которая представляет из себя разновидность **стекинга**, описаную в <a href='https://alexanderdyakonov.wordpress.com/2017/03/10/c%D1%82%D0%B5%D0%BA%D0%B8%D0%BD%D0%B3-stacking-%D0%B8-%D0%B1%D0%BB%D0%B5%D0%BD%D0%B4%D0%B8%D0%BD%D0%B3-blending/'>статье</a> на сайте **Александра Дьяконова**.   
   
А именно:  
- делим обучающую выборку на две части
- на первой настраиваем два алгоритма и делаем предсказания на второй
- получаем два метапризнака
- таким же образом получаем два метапризнака на тестовой выборке
- далее, на полученных метапризнаках из второй части обучающей выборки настраиваем линейный классификатор и делаем предсказания на тестовых метапризнаках

In [86]:
import xgboost as xgb

In [87]:
%%time
X_train_sparse_ohe_tfidf, y_train_ohe, X_test_sparse_ohe_tfidf = feature_engineering_df_OHE_Tfidf(train_df, test_df)

Wall time: 3min 12s


In [88]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_sparse_ohe_tfidf, y_train_ohe, 
                                                      test_size=0.5, 
                                                      random_state=17, stratify=y_train_ohe)

In [89]:
%%time
sgd_logit = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
sgd_pred_proba = sgd_logit.fit(X_train, y_train).predict_proba(X_valid)[:, 1]

Wall time: 334 ms


In [90]:
%%time
# инициализируем параметры
params = {
    'objective': 'binary:logistic',
    'max_depth': 11,
    'learning_rate': 0.5,
    'silent': 1.0,
    'n_estimators': 300
}

xgb_classifier = xgb.XGBClassifier(n_jobs=-1, random_state=17, **params)
xgb_pred_proba = xgb_classifier.fit(X_train, y_train).predict_proba(X_valid)[:, 1]

Wall time: 1min 8s


делаем новые **метапризнаки** и обучаем на них **SGDClassifier**

In [91]:
X_metafeature = np.hstack([sgd_pred_proba.reshape((-1, 1)), xgb_pred_proba.reshape((-1, 1))])

In [92]:
%%time
X_metafeature_test = np.hstack([sgd_logit.fit(X_train, y_train).predict_proba(X_test_sparse_ohe_tfidf)[:, 1].reshape((-1, 1)), 
                            xgb_classifier.fit(X_train, y_train).predict_proba(X_test_sparse_ohe_tfidf)[:, 1].reshape((-1, 1))])

Wall time: 1min 9s


In [93]:
%%time
meta_sgd_logit = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
test_pred_proba = meta_sgd_logit.fit(X_metafeature, y_valid).predict_proba(X_metafeature_test)[:, 1]
    
write_to_submission_file(test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_allfeatures_OHE_bal_tfidf_meta.csv'))

Wall time: 961 ms


**Kaggle: 0.94406**    

Попробуем **другую комбинацию** использования данных на аналогичном **блендинге**.    
Для компактности соберем все в **одну ячейку** и организуем **измерение времени** отдельных этапов.

In [94]:
import datetime

In [95]:
start_time = datetime.datetime.now() # фиксируем время для измерения времени этапа

sgd_logit = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
sgd_pred_proba = sgd_logit.fit(X_train, y_train).predict_proba(X_valid)[:, 1]

print('Time for sgd_pred_proba:', datetime.datetime.now() - start_time) # время этапа
start_time = datetime.datetime.now() # фиксируем время для измерения времени этапа

# инициализируем параметры
params = {
    'objective': 'binary:logistic',
    'max_depth': 11,
    'learning_rate': 0.5,
    'silent': 1.0,
    'n_estimators': 300
}

xgb_classifier = xgb.XGBClassifier(n_jobs=-1, random_state=17, **params)
xgb_pred_proba = xgb_classifier.fit(X_train, y_train).predict_proba(X_valid)[:, 1]
print('Time for xgb_pred_proba:', datetime.datetime.now() - start_time) # время этапа

X_metafeature = np.hstack([sgd_pred_proba.reshape((-1, 1)), xgb_pred_proba.reshape((-1, 1))])

start_time = datetime.datetime.now() # фиксируем время для измерения времени этапа

# вычисляем метапризнаки на тестовой выборке
X_metafeature_test = np.hstack([sgd_logit.fit(X_train_sparse_ohe_tfidf, y_train_ohe).predict_proba(X_test_sparse_ohe_tfidf)[:, 1].reshape((-1, 1)), 
                            xgb_classifier.fit(X_train_sparse_ohe_tfidf, y_train_ohe).predict_proba(X_test_sparse_ohe_tfidf)[:, 1].reshape((-1, 1))])

print('Time for X_metafeature_test:', datetime.datetime.now() - start_time) # время этапа
start_time = datetime.datetime.now() # фиксируем время для измерения времени этапа

meta_sgd_logit = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
test_pred_proba = meta_sgd_logit.fit(X_metafeature, y_valid).predict_proba(X_metafeature_test)[:, 1]
    
write_to_submission_file(test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_allfeatures_OHE_bal_tfidf_meta1.csv'))
print('Time for write_to_submission_file:', datetime.datetime.now() - start_time) # время этапа

Time for sgd_pred_proba: 0:00:00.358480
Time for xgb_pred_proba: 0:01:10.550903
Time for X_metafeature_test: 0:02:09.167243
Time for write_to_submission_file: 0:00:00.879340


**Kaggle: 0.94638**

In [96]:
start_time = datetime.datetime.now() # фиксируем время для измерения времени этапа

sgd_logit = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
sgd_pred_proba = sgd_logit.fit(X_train, y_train).predict_proba(X_valid)[:, 1]

print('Time for sgd_pred_proba:', datetime.datetime.now() - start_time) # время этапа
start_time = datetime.datetime.now() # фиксируем время для измерения времени этапа

# инициализируем параметры
params = {
    'objective': 'binary:logistic',
    'max_depth': 7,
    'learning_rate': 0.5,
    'silent': 1.0,
    'n_estimators': 170
}

xgb_classifier = xgb.XGBClassifier(n_jobs=-1, random_state=17, **params)
xgb_pred_proba = xgb_classifier.fit(X_train, y_train).predict_proba(X_valid)[:, 1]
print('Time for xgb_pred_proba:', datetime.datetime.now() - start_time) # время этапа

X_metafeature = np.hstack([sgd_pred_proba.reshape((-1, 1)), xgb_pred_proba.reshape((-1, 1))])

start_time = datetime.datetime.now() # фиксируем время для измерения времени этапа

# вычисляем метапризнаки на тестовой выборке
X_metafeature_test = np.hstack([sgd_logit.fit(X_train_sparse_ohe_tfidf, y_train_ohe).predict_proba(X_test_sparse_ohe_tfidf)[:, 1].reshape((-1, 1)), 
                            xgb_classifier.fit(X_train_sparse_ohe_tfidf, y_train_ohe).predict_proba(X_test_sparse_ohe_tfidf)[:, 1].reshape((-1, 1))])

print('Time for X_metafeature_test:', datetime.datetime.now() - start_time) # время этапа
start_time = datetime.datetime.now() # фиксируем время для измерения времени этапа

meta_sgd_logit = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
test_pred_proba = meta_sgd_logit.fit(X_metafeature, y_valid).predict_proba(X_metafeature_test)[:, 1]
    
write_to_submission_file(test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_allfeatures_OHE_bal_tfidf_meta1.csv'))
print('Time for write_to_submission_file:', datetime.datetime.now() - start_time) # время этапа

Time for sgd_pred_proba: 0:00:00.360444
Time for xgb_pred_proba: 0:00:29.772338
Time for X_metafeature_test: 0:00:52.549302
Time for write_to_submission_file: 0:00:00.779422


**Kaggle: 0.94642**

Уменьшим максимальную глубину деревьев в **xgboost**

In [97]:
start_time = datetime.datetime.now() # фиксируем время для измерения времени этапа

sgd_logit = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
sgd_pred_proba = sgd_logit.fit(X_train, y_train).predict_proba(X_valid)[:, 1]

print('Time for sgd_pred_proba:', datetime.datetime.now() - start_time) # время этапа
start_time = datetime.datetime.now() # фиксируем время для измерения времени этапа

# инициализируем параметры
params = {
    'objective': 'binary:logistic',
    'max_depth': 5,
    'learning_rate': 0.5,
    'silent': 1.0,
    'n_estimators': 170
}

xgb_classifier = xgb.XGBClassifier(n_jobs=-1, random_state=17, **params)
xgb_pred_proba = xgb_classifier.fit(X_train, y_train).predict_proba(X_valid)[:, 1]
print('Time for xgb_pred_proba:', datetime.datetime.now() - start_time) # время этапа

X_metafeature = np.hstack([sgd_pred_proba.reshape((-1, 1)), xgb_pred_proba.reshape((-1, 1))])

start_time = datetime.datetime.now() # фиксируем время для измерения времени этапа

# вычисляем метапризнаки на тестовой выборке
X_metafeature_test = np.hstack([sgd_logit.fit(X_train_sparse_ohe_tfidf, 
                                              y_train_ohe).predict_proba(X_test_sparse_ohe_tfidf)[:, 1].reshape((-1, 1)), 
                                xgb_classifier.fit(X_train_sparse_ohe_tfidf, 
                                                   y_train_ohe).predict_proba(X_test_sparse_ohe_tfidf)[:, 1].reshape((-1, 1))])

print('Time for X_metafeature_test:', datetime.datetime.now() - start_time) # время этапа
start_time = datetime.datetime.now() # фиксируем время для измерения времени этапа

meta_sgd_logit = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
test_pred_proba = meta_sgd_logit.fit(X_metafeature, y_valid).predict_proba(X_metafeature_test)[:, 1]
    
write_to_submission_file(test_pred_proba, os.path.join(PATH_TO_DATA, 
                                                       '[YDF & MIPT]_Coursera_allfeatures_OHE_bal_tfidf_meta2.csv'))
print('Time for write_to_submission_file:', datetime.datetime.now() - start_time) # время этапа

Time for sgd_pred_proba: 0:00:00.354459
Time for xgb_pred_proba: 0:00:24.615989
Time for X_metafeature_test: 0:00:41.172088
Time for write_to_submission_file: 0:00:00.767043


**Kaggle: 0.94739**    
Неплохо, результат **вырос** при **уменьшении глубины** деревьев (вероятно, из-за уменьшения переобучения), но все же по-прежнему немного **уступает** самому лучшему результату, полученному выше на **SGDCclassifier**.   
Тем не менее опыт интересный.

**РЕЗЮМЕ:**   
Итак, финальный результат этой недели - **результат на Kaggle: 0.94773**.   
На сегодня это **63-я** позиция на **Public Leaderboard**, команда в соревновании - **[YDF & MIPT] Dmitry Shereshevskiy**   
Бенчмарки этой недели -  **"sgd_logit_benchmark.csv"** и **"Logit +3 features"** - **побиты**.